<a href="https://colab.research.google.com/github/Erssa001/ECGR_4105/blob/main/HW3_P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import metrics

import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/My Drive/2023-2024/ECGR_4105/cancer.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))

def binary_map(x):
    return x.map({'B':  0, 'M': 1})

dataset[['diagnosis']] = dataset[['diagnosis']].apply(binary_map).values

dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
X = dataset.iloc[:, 2:32].values
Y = dataset.iloc[:, 1].values

# Input data is standardized to fit a normal curve.
sc_X = StandardScaler()
X = sc_X.fit_transform(X)


In [ ]:
n_components = 15
pca = PCA(n_components=n_components)
principalComponents = pca.fit_transform(X)

#Data is split into trainng (80%) and validation sets (20%)
X_train, X_test, Y_train, Y_test = train_test_split(principalComponents, Y, test_size = 0.20, random_state = 0)

column_names = [f'principal component {i+1}' for i in range(n_components)]
principalDf = pd.DataFrame(data=principalComponents, columns=column_names)
principalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14,principal component 15
0,9.192837,1.948583,-1.123166,3.633731,-1.195110,1.411424,2.159370,-0.398407,-0.157118,-0.877402,0.262956,-0.859015,0.103387,-0.690803,-0.601773
1,2.387802,-3.768172,-0.529293,1.118264,0.621775,0.028656,0.013358,0.240989,-0.711905,1.106995,0.813120,0.157922,-0.943529,-0.653477,0.008952
2,5.733896,-1.075174,-0.551748,0.912083,-0.177086,0.541452,-0.668166,0.097374,0.024066,0.454275,-0.605604,0.124387,-0.410626,0.016679,0.483410
3,7.122953,10.275589,-3.232790,0.152547,-2.960878,3.053422,1.429911,1.059566,-1.405440,-1.116975,-1.151513,1.011313,-0.933270,-0.487421,-0.168893
4,3.935302,-1.948072,1.389767,2.940639,0.546747,-1.226495,-0.936213,0.636376,-0.263805,0.377704,0.651360,-0.110515,0.387948,-0.539180,0.310319


In [ ]:
#Make an instance classifier of the object LogisticRegression and give random_state =  0
#classifier = LogisticRegression(random_state=0) #Uncomment this line for Part A.
classifier = LogisticRegression(random_state=0,penalty='l2',C=.5) #Uncomment this line for Part B.
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)

In [ ]:
#Next the scores for different validation metrics are displayed.

P = metrics.precision_score(Y_test, Y_pred)
R = metrics.recall_score(Y_test, Y_pred)
F1 = 2 * (P * R) / (P + R)
print(f"Accuracy: {100 * metrics.accuracy_score(Y_test, Y_pred):.2f}%")
print(f"Precision: {100 * P:.2f}%")
print(f"Recall: {100 * R:.2f}%")
print(f"F1 Score: {F1:.3f}")

Accuracy: 96.49%
Precision: 95.74%
Recall: 95.74%
F1 Score: 0.957
